examples from https://morvanzhou.github.io/

should use coldfoding to read

# threading

In [1]:
import threading

In [2]:
threading.active_count(), threading.enumerate(), threading.current_thread()

## join 等待

In [3]:
import time
import threading

In [4]:
# main 執行完，T1還在執行，在jupyter 就被咖掉了
def T1_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1)  # T1會執行比較久
    print("T1 finish\n")

def T2_job():
    print("T2 start\n")
    print("T2 finish\n")

thread_1 = threading.Thread(target=T1_job, name='T1')  # 可以命名thread
thread_2 = threading.Thread(target=T2_job, name='T2')

thread_1.start() 
thread_2.start() 

print("all done\n")  

T1 start

T2 start

T2 finish

all done



In [5]:
# 加上join 等待
def T1_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1)  # T1會執行比較久
    print("T1 finish\n")

def T2_job():
    print("T2 start\n")
    print("T2 finish\n")

thread_1 = threading.Thread(target=T1_job, name='T1')
thread_2 = threading.Thread(target=T2_job, name='T2')

thread_1.start() 
thread_2.start() 

thread_1.join() # 等T1執行完才繼續下面的程式
print("all done\n")

T1 start

T2 start

T2 finish

T1 finish

T1 finish

all done



## Queue 可以有return

In [6]:
import queue
import threading
import numpy as np

In [7]:
def job(l, q):
    for i in range(len(l)):
        l[i] = l[i]**2
    q.put(l)  # return 存進這

In [8]:
# multithreading
num_threads = 4

q = queue.Queue()
threads = []
result = []
data = np.array([[1, 2, 3], 
                 [3, 4, 5], 
                 [4, 4, 4], 
                 [5, 5, 5]])


for i in range(num_threads):   
    t = threading.Thread(target=job,args=(data[i], q))
    t.start()  # 開始
    threads.append(t)
    
# 等待全部線程執行完    
for i in range(num_threads):    
    t.join()  
    
# 取出result
for i in range(num_threads):
    result.append(q.get())

result = np.array(result)
print(result)

[[ 1  4  9]
 [ 9 16 25]
 [16 16 16]
 [25 25 25]]


## GIL (Global Interpreter Lock)

拜它所賜，在python開多線程不一定會比較快(可能還比較慢)，在I/O比較多的case，才會比較快

In [9]:
import copy
import time
import threading

from queue import Queue

In [10]:
def job(l, q):
    res = sum(l)
    q.put(res)

In [11]:
def multithreading(l, num_threads=10):
    q = Queue()
    threads = []
    for i in range(num_threads):
        t = threading.Thread(target=job, args=(copy.copy(l), q), name='T%i' % i)     
        t.start()
        threads.append(t)

    [t.join() for t in threads]
    
    total = 0
    for _ in range(num_threads):
        total += q.get()


def normal(l):
    total = sum(l)

In [12]:
l = list(range(1000000))

s_t = time.time()
normal(l*4)
print(f'normal: {time.time()-s_t: .2f}')

s_t = time.time()
multithreading(l, num_threads=10)
print(f'10 threading: {time.time()-s_t:.2f}')

s_t = time.time()
multithreading(l, num_threads=2)
print(f'2 threading: {time.time()-s_t:.2f}')

normal:  0.10
10 threading: 0.24
2 threading: 0.05


## lock

In [20]:
# without lock, the result would be messy
import threading


def job1():
    global A
    for i in range(10):
        A += 1
        print('job1', A)


def job2():
    global A
    for i in range(10):
        A += 10
        print('job2', A)


A = 0

t1 = threading.Thread(target=job1)
t2 = threading.Thread(target=job2)

t1.start()
t2.start()

t1.join()
t2.join()

job1 1
job1 2
job1 3
job1 4
job1 5
job1 6
job1 7job2 17
job2
job1 28
job1 29
job1 30
 27
job2 40
job2 50
job2 60
job2 70
job2 80
job2 90
job2 100
job2 110


In [14]:
# with lock
import threading


def job1():
    global A, lock
    lock.acquire()
    for i in range(10):
        A += 1
        print('job1', A)
    lock.release()


def job2():
    global A, lock
    lock.acquire()
    for i in range(10):
        A += 10
        print('job2', A)
    lock.release()


A = 0

lock = threading.Lock()

t1 = threading.Thread(target=job1)
t2 = threading.Thread(target=job2)

t1.start()
t2.start()

t1.join()
t2.join()

job1 1
job1 2
job1 3
job1 4
job1 5
job1 6
job1 7
job1 8
job1 9
job1 10
job2 20
job2 30
job2 40
job2 50
job2 60
job2 70
job2 80
job2 90
job2 100
job2 110
